In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from Erwin.datasets.utils import iterators
from Erwin.datasets import Dataset

In [79]:
y_pred = tf.placeholder(shape=(3,4), dtype=tf.float32)
y_true = tf.placeholder(shape=(3,4), dtype=tf.float32)
weights = tf.placeholder(shape=(3,4), dtype=tf.int32)
acc, acc_update = tf.contrib.metrics.streaming_accuracy(y_true, y_pred, weights)

In [80]:
#y_pred_numpy = np.asarray([[0, 0, 1], [0, 1, 0]], dtype=np.float32)
#y_true_numpy = np.asarray([[0, 0, 1], [1, 0, 1]], dtype=np.float32)
#weights_numpy = np.asarray([[1, 1, 1], [1, 1, 0]], dtype=np.int32)
y_pred_numpy = np.random.randint(0, 2, size=(10, 3, 4)).astype(np.float32)
y_true_numpy = np.random.randint(0, 2, size=(10, 3, 4)).astype(np.float32)
weights_numpy = np.random.binomial(2, p=0.1, size=(10, 3, 4))
y_pred_numpy.shape, weights_numpy.shape

((10, 3, 4), (10, 3, 4))

In [81]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [82]:
for y_p, y_t, w in zip(y_pred_numpy, y_true_numpy, weights_numpy[:, :, :]):
    res = sess.run([acc_update], feed_dict={y_pred: y_p, y_true: y_t, weights: w})
    print(res)

[0.66666669]
[0.33333334]
[0.375]
[0.5]
[0.5]
[0.53846157]
[0.625]
[0.55000001]
[0.54545456]
[0.52173913]


In [84]:
loss = tf.placeholder(dtype=tf.float32)
st_loss, loss_update = tf.contrib.metrics.streaming_concat(loss)

ValueError: `values` must have known statically known rank